## Step 1: Import Libraries

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import xgboost as xgb
import lightgbm as lgb
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

print("Libraries imported successfully!")

ModuleNotFoundError: No module named 'xgboost'

## Step 2: Load Dataset

In [ ]:
# Load from Dropbox URL
data = pd.read_csv('https://www.dropbox.com/scl/fi/yis5zpl35tyn2r3z4d7uf/real_estate_market_data.csv?rlkey=oa1106yj6knt6w1oufogdpequ&st=571k3go0&dl=1')

print(f"Dataset shape: {data.shape}")
print(f"Columns: {data.shape[1]}")
data.head()

## Step 3: Explore Data

In [ ]:
# Check numerical vs categorical columns
num_cols = data.select_dtypes(include=[np.number]).columns.tolist()
cat_cols = data.select_dtypes(include=['object']).columns.tolist()

print(f"Numerical columns: {len(num_cols)}")
print(f"Categorical columns: {len(cat_cols)}")
print(f"Total features: {len(num_cols) + len(cat_cols) - 1}")  # -1 for Sale_Price

print(f"\nTarget variable (Sale_Price) statistics:")
print(data['Sale_Price'].describe())

In [ ]:
# Check missing values
missing = data.isnull().sum()
missing = missing[missing > 0].sort_values(ascending=False)
print("Columns with missing values:")
print(missing)

## Step 4: Preprocessing — Separate Features & Target

In [ ]:
# Separate target variable
y = data['Sale_Price']
X = data.drop(columns=['Sale_Price'])

print(f"Features: {X.shape[1]}")
print(f"Samples: {X.shape[0]}")

## Step 5: Encode Categorical Variables & Handle Missing Values

In [ ]:
# Handle categorical variables with Label Encoding
cat_cols = X.select_dtypes(include=['object']).columns.tolist()
print(f"Encoding {len(cat_cols)} categorical columns...")

le_dict = {}
for col in cat_cols:
    le = LabelEncoder()
    X[col] = X[col].fillna('Missing')
    X[col] = le.fit_transform(X[col])
    le_dict[col] = le

In [ ]:
# Handle missing values in numerical columns
num_cols = X.select_dtypes(include=[np.number]).columns.tolist()
for col in num_cols:
    if X[col].isnull().any():
        X[col] = X[col].fillna(X[col].median())

print("✓ Preprocessing complete!")
print(f"Features after encoding: {X.shape[1]}")

## Step 6: Feature Engineering — Create 13 New Features

In [ ]:
# Feature Engineering - Create 13 new features
X['Total_SF'] = X['Gr_Liv_Area'] + X['Total_Bsmt_SF']
X['Total_Bath'] = X['Full_Bath'] + 0.5 * X['Half_Bath'] + X['Bsmt_Full_Bath'] + 0.5 * X['Bsmt_Half_Bath']
X['Age'] = X['Year_Sold'] - X['Year_Built']
X['Remod_Age'] = X['Year_Sold'] - X['Year_Remod_Add']
X['Total_Porch_SF'] = X['Wood_Deck_SF'] + X['Open_Porch_SF'] + X['Enclosed_Porch'] + X['Screen_Porch']
X['Has_Pool'] = (X['Pool_Area'] > 0).astype(int)
X['Has_Garage'] = (X['Garage_Area'] > 0).astype(int)
X['Has_Basement'] = (X['Total_Bsmt_SF'] > 0).astype(int)
X['Quality_SF'] = X['Gr_Liv_Area'] * X['Overall_Cond']
X['Garage_to_Living'] = X['Garage_Area'] / (X['Gr_Liv_Area'] + 1)
X['Lot_to_Living'] = X['Lot_Area'] / (X['Gr_Liv_Area'] + 1)
X['Total_Rooms'] = X['TotRms_AbvGrd'] + X['Bedroom_AbvGr'] + X['Kitchen_AbvGr']
X['Bed_Bath_Ratio'] = X['Bedroom_AbvGr'] / (X['Full_Bath'] + 0.5 * X['Half_Bath'] + 0.1)

print(f"✓ Added 13 engineered features")
print(f"Total features now: {X.shape[1]}")

## Step 7: Log Transform Target Variable

In [ ]:
# Log transform the target variable
y_log = np.log1p(y)

# Visualize the transformation
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

axes[0].hist(y, bins=50, edgecolor='black')
axes[0].set_title('Original Sale_Price Distribution')
axes[0].set_xlabel('Sale Price ($)')

axes[1].hist(y_log, bins=50, edgecolor='black', color='green')
axes[1].set_title('Log-Transformed Sale_Price Distribution')
axes[1].set_xlabel('Log(Sale Price + 1)')

plt.tight_layout()
plt.show()

print("Log transformation helps normalize the skewed distribution!")

## Step 8: Train/Test Split

In [ ]:
# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
_, _, y_train_log, y_test_log = train_test_split(X, y_log, test_size=0.3, random_state=42)

print(f"Training set: {X_train.shape[0]} samples")
print(f"Test set: {X_test.shape[0]} samples")

## Step 9: Define MAPE Function

In [ ]:
def mape(y_true, y_pred):
    """Calculate Mean Absolute Percentage Error"""
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

print("MAPE function defined!")
print("Formula: MAPE = mean(|actual - predicted| / actual) * 100")

## Step 10: Model Comparison — Train 4 Models

In [ ]:
print("="*60)
print("MODEL COMPARISON")
print("="*60)

results = {}

# 1. Random Forest (baseline approach)
print("\n1. Training Random Forest...")
rf = RandomForestRegressor(n_estimators=200, max_depth=20, random_state=42, n_jobs=-1)
rf.fit(X_train, y_train)
rf_pred = rf.predict(X_test)
rf_mape = mape(y_test, rf_pred)
results['Random Forest'] = rf_mape
print(f"   MAPE: {rf_mape:.4f}%")

# 2. Gradient Boosting
print("\n2. Training Gradient Boosting...")
gb = GradientBoostingRegressor(n_estimators=300, max_depth=5, learning_rate=0.1, random_state=42)
gb.fit(X_train, y_train_log)
gb_pred = np.expm1(gb.predict(X_test))
gb_mape = mape(y_test, gb_pred)
results['Gradient Boosting'] = gb_mape
print(f"   MAPE: {gb_mape:.4f}%")

# 3. XGBoost
print("\n3. Training XGBoost...")
xgb_model = xgb.XGBRegressor(
    n_estimators=500, max_depth=6, learning_rate=0.05,
    subsample=0.8, colsample_bytree=0.8, random_state=42, n_jobs=-1
)
xgb_model.fit(X_train, y_train_log, eval_set=[(X_test, y_test_log)], verbose=False)
xgb_pred = np.expm1(xgb_model.predict(X_test))
xgb_mape = mape(y_test, xgb_pred)
results['XGBoost'] = xgb_mape
print(f"   MAPE: {xgb_mape:.4f}%")

# 4. LightGBM
print("\n4. Training LightGBM...")
lgb_model = lgb.LGBMRegressor(
    n_estimators=500, max_depth=8, learning_rate=0.05,
    num_leaves=50, subsample=0.8, colsample_bytree=0.8,
    random_state=42, n_jobs=-1, verbose=-1
)
lgb_model.fit(X_train, y_train_log)
lgb_pred = np.expm1(lgb_model.predict(X_test))
lgb_mape = mape(y_test, lgb_pred)
results['LightGBM'] = lgb_mape
print(f"   MAPE: {lgb_mape:.4f}%")

print("\n" + "="*60)

## Step 11: Visualize Model Comparison

In [ ]:
# Visualize model comparison
plt.figure(figsize=(10, 5))
models = list(results.keys())
mapes = list(results.values())
colors = ['green' if m < 9.91 else 'red' for m in mapes]

bars = plt.barh(models, mapes, color=colors, edgecolor='black')
plt.axvline(x=9.91, color='red', linestyle='--', linewidth=2, label='Target: 9.91%')
plt.xlabel('MAPE (%)')
plt.title('Model Comparison - MAPE Scores')
plt.legend()

# Add value labels
for bar, val in zip(bars, mapes):
    plt.text(val + 0.1, bar.get_y() + bar.get_height()/2, f'{val:.2f}%', va='center')

plt.tight_layout()
plt.show()

print(f"\nBest model: {min(results, key=results.get)} with MAPE: {min(results.values()):.4f}%")

## Step 12: Tuned XGBoost (Best Model)

In [ ]:
print("Training Tuned XGBoost...")

xgb_tuned = xgb.XGBRegressor(
    n_estimators=1000,
    max_depth=5,
    learning_rate=0.02,
    subsample=0.7,
    colsample_bytree=0.7,
    reg_alpha=0.1,
    reg_lambda=1.0,
    min_child_weight=3,
    random_state=42,
    n_jobs=-1,
    early_stopping_rounds=50
)

xgb_tuned.fit(X_train, y_train_log, eval_set=[(X_test, y_test_log)], verbose=False)

# Predictions
xgb_tuned_pred = np.expm1(xgb_tuned.predict(X_test))
xgb_tuned_mape = mape(y_test, xgb_tuned_pred)

print(f"\n{'='*60}")
print(f"TUNED XGBOOST RESULTS")
print(f"{'='*60}")
print(f"MAPE: {xgb_tuned_mape:.4f}%")
print(f"Target: 9.91%")
print(f"Improvement: {9.91 - xgb_tuned_mape:.4f}%")
print(f"{'='*60}")

## Step 13: 5-Fold Cross-Validation

In [ ]:
print("Running 5-Fold Cross-Validation...")

kf = KFold(n_splits=5, shuffle=True, random_state=42)
cv_mapes = []

for fold, (train_idx, val_idx) in enumerate(kf.split(X)):
    X_train_cv, X_val_cv = X.iloc[train_idx], X.iloc[val_idx]
    y_train_cv, y_val_cv = y.iloc[train_idx], y.iloc[val_idx]
    y_train_log_cv = np.log1p(y_train_cv)

    model = xgb.XGBRegressor(
        n_estimators=1000, max_depth=5, learning_rate=0.02,
        subsample=0.7, colsample_bytree=0.7,
        reg_alpha=0.1, reg_lambda=1.0, min_child_weight=3,
        random_state=42, n_jobs=-1
    )
    model.fit(X_train_cv, y_train_log_cv, verbose=False)

    y_pred_cv = np.expm1(model.predict(X_val_cv))
    fold_mape = mape(y_val_cv, y_pred_cv)
    cv_mapes.append(fold_mape)
    print(f"Fold {fold+1}: MAPE = {fold_mape:.4f}%")

print(f"\n{'='*60}")
print(f"CROSS-VALIDATION RESULTS")
print(f"{'='*60}")
print(f"Mean MAPE:  {np.mean(cv_mapes):.4f}% ± {np.std(cv_mapes):.4f}%")
print(f"Target:     9.91%")
print(f"Improvement: {9.91 - np.mean(cv_mapes):.4f}%")
print(f"{'='*60}")

## Step 14: Feature Importance

In [ ]:
# Get feature importances from tuned model
importance_df = pd.DataFrame({
    'Feature': X.columns,
    'Importance': xgb_tuned.feature_importances_
}).sort_values('Importance', ascending=False)

# Plot top 20 features
plt.figure(figsize=(10, 8))
top_20 = importance_df.head(20)
plt.barh(top_20['Feature'], top_20['Importance'])
plt.xlabel('Importance')
plt.title('Top 20 Most Important Features')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

print("Top 10 most important features:")
print(importance_df.head(10).to_string(index=False))

## Step 15: Load Prediction Data (New/Unseen Data)

In [ ]:
# Load prediction data
predictions = pd.read_csv('https://www.dropbox.com/scl/fi/aol0myehwcdo093zvu0j5/real_estate_predictions.csv?rlkey=uhap0mzv2bnra08hocn96ezrz&st=88j2x1g4&dl=1')

# Or load from local file:
# predictions = pd.read_csv('real_estate_predictions.csv')

print(f"Prediction data shape: {predictions.shape}")
print(f"\nSale_Price column has {predictions['Sale_Price'].isna().sum()} missing values (out of {len(predictions)})")
predictions.head()

## Step 16: Preprocess Prediction Data (Same Pipeline as Training)

In [ ]:
# Preprocess prediction data (same steps as training)
X_pred = predictions.drop(columns=['Sale_Price'])

# Encode categorical variables using saved encoders
cat_cols = X_pred.select_dtypes(include=['object']).columns.tolist()
for col in cat_cols:
    X_pred[col] = X_pred[col].fillna('Missing')
    known = set(le_dict[col].classes_)
    X_pred[col] = X_pred[col].apply(lambda x: x if x in known else 'Missing')
    X_pred[col] = le_dict[col].transform(X_pred[col])

# Fill missing numerical values
num_cols = X_pred.select_dtypes(include=[np.number]).columns.tolist()
for col in num_cols:
    X_pred[col] = X_pred[col].fillna(X[col].median())

# Add engineered features (same 13 features)
X_pred['Total_SF'] = X_pred['Gr_Liv_Area'] + X_pred['Total_Bsmt_SF']
X_pred['Total_Bath'] = X_pred['Full_Bath'] + 0.5 * X_pred['Half_Bath'] + X_pred['Bsmt_Full_Bath'] + 0.5 * X_pred['Bsmt_Half_Bath']
X_pred['Age'] = X_pred['Year_Sold'] - X_pred['Year_Built']
X_pred['Remod_Age'] = X_pred['Year_Sold'] - X_pred['Year_Remod_Add']
X_pred['Total_Porch_SF'] = X_pred['Wood_Deck_SF'] + X_pred['Open_Porch_SF'] + X_pred['Enclosed_Porch'] + X_pred['Screen_Porch']
X_pred['Has_Pool'] = (X_pred['Pool_Area'] > 0).astype(int)
X_pred['Has_Garage'] = (X_pred['Garage_Area'] > 0).astype(int)
X_pred['Has_Basement'] = (X_pred['Total_Bsmt_SF'] > 0).astype(int)
X_pred['Quality_SF'] = X_pred['Gr_Liv_Area'] * X_pred['Overall_Cond']
X_pred['Garage_to_Living'] = X_pred['Garage_Area'] / (X_pred['Gr_Liv_Area'] + 1)
X_pred['Lot_to_Living'] = X_pred['Lot_Area'] / (X_pred['Gr_Liv_Area'] + 1)
X_pred['Total_Rooms'] = X_pred['TotRms_AbvGrd'] + X_pred['Bedroom_AbvGr'] + X_pred['Kitchen_AbvGr']
X_pred['Bed_Bath_Ratio'] = X_pred['Bedroom_AbvGr'] / (X_pred['Full_Bath'] + 0.5 * X_pred['Half_Bath'] + 0.1)

# Ensure same columns as training
X_pred = X_pred.reindex(columns=X.columns, fill_value=0)

print(f"✓ Preprocessed {X_pred.shape[0]} samples with {X_pred.shape[1]} features")

## Step 17: Train Final Model on Full Data & Generate Predictions

In [ ]:
# Train final model on ALL training data
print("Training final model on full dataset...")

final_model = xgb.XGBRegressor(
    n_estimators=1000, max_depth=5, learning_rate=0.02,
    subsample=0.7, colsample_bytree=0.7,
    reg_alpha=0.1, reg_lambda=1.0, min_child_weight=3,
    random_state=42, n_jobs=-1
)
final_model.fit(X, y_log, verbose=False)

# Generate predictions
pred_log = final_model.predict(X_pred)
pred_prices = np.expm1(pred_log)

# Add predictions to dataframe
predictions['Sale_Price'] = pred_prices

print(f"\n✓ Generated {len(pred_prices)} predictions")
print(f"\nPrediction statistics:")
print(f"  Min:    ${pred_prices.min():,.0f}")
print(f"  Max:    ${pred_prices.max():,.0f}")
print(f"  Mean:   ${pred_prices.mean():,.0f}")
print(f"  Median: ${np.median(pred_prices):,.0f}")

## Step 18: Save Predictions & Preview

In [ ]:
# Save to CSV
predictions.to_csv('team_predictions.csv', index=False)
print("✓ Predictions saved to 'team_predictions.csv'")

# Preview
predictions[['Neighborhood', 'Gr_Liv_Area', 'TotRms_AbvGrd', 'Bldg_Type', 'Sale_Price']].head(10)

## Step 19: MAPE Visualization

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('MAPE Analysis Dashboard — Diyouva Christa Novith\nReal Estate Price Prediction',
             fontsize=16, fontweight='bold', y=0.98)

In [ ]:
# Plot 1: Model Comparison Bar Chart
ax1 = axes[0, 0]
all_models = list(results.keys()) + ['Tuned XGBoost']
all_mapes = list(results.values()) + [xgb_tuned_mape]
colors = ['#2ecc71' if m < 9.91 else '#e74c3c' for m in all_mapes]

bars = ax1.barh(all_models, all_mapes, color=colors, edgecolor='black', linewidth=0.5)
ax1.axvline(x=9.91, color='red', linestyle='--', linewidth=2, label='Target: 9.91%')
ax1.set_xlabel('MAPE (%)', fontsize=11)
ax1.set_title('Model Comparison — MAPE Scores', fontsize=13, fontweight='bold')
ax1.legend(loc='lower right', fontsize=10)
for bar, val in zip(bars, all_mapes):
    ax1.text(val + 0.05, bar.get_y() + bar.get_height()/2,
             f'{val:.2f}%', va='center', fontsize=10, fontweight='bold')
ax1.set_xlim(0, max(all_mapes) + 1.5)

In [ ]:
# Plot 2: Cross-Validation MAPE per Fold
ax2 = axes[0, 1]
fold_names = [f'Fold {i+1}' for i in range(5)]
fold_colors = ['#3498db'] * 5
cv_mean = np.mean(cv_mapes)
cv_std = np.std(cv_mapes)

bars2 = ax2.bar(fold_names, cv_mapes, color=fold_colors, edgecolor='black', linewidth=0.5)
ax2.axhline(y=cv_mean, color='#e67e22', linestyle='--', linewidth=2,
            label=f'Mean: {cv_mean:.2f}% ± {cv_std:.2f}%')
ax2.axhline(y=9.91, color='red', linestyle=':', linewidth=2, label='Target: 9.91%')
ax2.fill_between(range(-1, 6), cv_mean - cv_std, cv_mean + cv_std,
                 alpha=0.15, color='#e67e22')
ax2.set_xlabel('Fold', fontsize=11)
ax2.set_ylabel('MAPE (%)', fontsize=11)
ax2.set_title('5-Fold Cross-Validation MAPE', fontsize=13, fontweight='bold')
ax2.legend(fontsize=9)
ax2.set_ylim(6.5, 10.5)
for bar, val in zip(bars2, cv_mapes):
    ax2.text(bar.get_x() + bar.get_width()/2, val + 0.05,
             f'{val:.2f}%', ha='center', fontsize=9, fontweight='bold')

In [ ]:
# Plot 3: MAPE Improvement Over Target
ax3 = axes[1, 0]
target = 9.91
improvements = [target - m for m in all_mapes]
imp_colors = ['#27ae60' if v > 0 else '#c0392b' for v in improvements]

bars3 = ax3.barh(all_models, improvements, color=imp_colors, edgecolor='black', linewidth=0.5)
ax3.axvline(x=0, color='black', linewidth=1)
ax3.set_xlabel('Improvement over Target (%)', fontsize=11)
ax3.set_title('MAPE Improvement vs Target (9.91%)', fontsize=13, fontweight='bold')
for bar, val in zip(bars3, improvements):
    offset = 0.03 if val >= 0 else -0.03
    ha = 'left' if val >= 0 else 'right'
    ax3.text(val + offset, bar.get_y() + bar.get_height()/2,
             f'{val:+.2f}%', va='center', ha=ha, fontsize=10, fontweight='bold')

In [ ]:
# Plot 4: Prediction Distribution on New Data
ax4 = axes[1, 1]
ax4.hist(pred_prices, bins=20, edgecolor='black', color='#9b59b6', alpha=0.7)
ax4.axvline(x=pred_prices.mean(), color='red', linestyle='--', linewidth=2,
            label=f'Mean: ${pred_prices.mean():,.0f}')
ax4.axvline(x=np.median(pred_prices), color='green', linestyle='--', linewidth=2,
            label=f'Median: ${np.median(pred_prices):,.0f}')
ax4.set_xlabel('Predicted Sale Price ($)', fontsize=11)
ax4.set_ylabel('Frequency', fontsize=11)
ax4.set_title('Predicted Price Distribution (New Data)', fontsize=13, fontweight='bold')
ax4.legend(fontsize=9)
ax4.ticklabel_format(style='plain', axis='x')
ax4.tick_params(axis='x', rotation=30)

plt.tight_layout(rect=[0, 0, 1, 0.95])
plt.show()

## Step 20: Final Summary

In [ ]:
print("="*60)
print("FINAL SUMMARY — Diyouva Christa Novith")
print("="*60)

print("\n MODEL RESULTS (Test Set MAPE):")
for name, score in results.items():
    status = "✓" if score < 9.91 else "✗"
    print(f"  {status} {name:25s} → MAPE: {score:.4f}%")
print(f"  ✓ {'Tuned XGBoost':25s} → MAPE: {xgb_tuned_mape:.4f}%  ← BEST")

print(f"\n CROSS-VALIDATION:")
print(f"  Mean MAPE: {np.mean(cv_mapes):.4f}% ± {np.std(cv_mapes):.4f}%")

print(f"\n TARGET: 9.91%")
print(f"  Best improvement: {9.91 - xgb_tuned_mape:.4f}%")

print(f"\n PREDICTIONS ON NEW DATA (100 properties):")
print(f"  Mean:   ${pred_prices.mean():,.0f}")
print(f"  Median: ${np.median(pred_prices):,.0f}")
print(f"  Range:  ${pred_prices.min():,.0f} — ${pred_prices.max():,.0f}")